In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
pd.set_option('display.max_columns', None)
from get_dataframes import GetDataframes
import hjson

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

- Importation de la dataframe contenant seulement les acteurs.
- Cette dataframe a déjà été nettoyé comme indiqué précédemment.

In [2]:
with open("config.hjson") as fp:
    config = hjson.load(fp)
datas = GetDataframes(config)
link = "actors_movies"
df = datas.get_dataframes(
    link,
    True
)

2023-11-10 14:50:36 INFO     TSV files already exist.
2023-11-10 14:50:36 INFO     Parquet loaded ! Importing actors_movies...
2023-11-10 14:50:36 INFO     Dataframe actors_movies ready to use!


- On veut afficher quels acteurs ont joué dans le plus de films pour chaque décénnie.

In [3]:
df.describe()

,titre_date_sortie,titre_duree,rating_avg,rating_votes,popularity,revenue,person_birthdate,person_index
count,32133.00,32133.00,32133.00,32133.00,32133.00,32133.00,32133.00,32133.00
mean,2001.21,112.75,7.05,95558.29,22.15,60139082.59,1758.71,2.54
std,16.56,20.52,0.53,189836.57,85.83,162618533.01,596.43,1.22
min,1960.00,63.00,6.30,5004.00,0.60,0.00,0.00,1.00
25%,1991.00,98.00,6.60,10302.00,8.30,0.00,1938.00,2.00
50%,2005.00,109.00,7.00,26503.00,11.60,3919254.00,1960.00,3.00
75%,2015.00,123.00,7.40,92338.00,18.48,41895491.00,1975.00,4.00
max,2023.00,210.00,9.30,2816055.00,3935.55,2923706026.00,2012.00,10.00


In [4]:
grouped_df = df.groupby(
    ['cuts', 'person_name'],
            observed=True
).size().reset_index(name='total_film_acteurs').sort_values(by="total_film_acteurs")

top_acteurs_decennie = grouped_df.groupby(
    'cuts',
            observed=True
).apply(lambda x: x.nlargest(1, 'total_film_acteurs')).reset_index(drop=True)

decennies = top_acteurs_decennie['cuts']
noms_acteurs = top_acteurs_decennie['person_name']
nombre_films = top_acteurs_decennie['total_film_acteurs']

fig = go.Figure(data=[
    go.Bar(
        x=nombre_films,
        y=decennies,
        orientation='h',
        marker=dict(
            color='#66cdaa',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=noms_acteurs,
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Acteur N°1 par Décennie et Nombre de Films Joués',
    xaxis_title='Nombre de Films Joués',
    yaxis_title='Décennie'
)

fig.show()


- On veut afficher quels acteurs ont joué dans le plus de films pour chaque genre.

In [5]:
actors_by_genre = df.explode('titre_genres').groupby(['person_name', 'titre_genres']).size().reset_index(name='count')
top_actors_by_genre = actors_by_genre.sort_values('count', ascending=False).drop_duplicates('titre_genres')[:10][::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_actors_by_genre['count'],
        y=top_actors_by_genre['titre_genres'],
        orientation='h',
        marker=dict(
            color='#006b3c',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_by_genre['person_name'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='white'
        )
    )
])

fig.update_layout(
    title='Acteurs les plus fréquemment associés aux top 10 des genres',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()



- On veut afficher quels acteurs ont joué dans le plus de films au total.

In [6]:
actors_film_count = df.groupby('person_name').size().reset_index(name='film_count')

top_actors_film_count = actors_film_count.sort_values('film_count', ascending=False).head(10)[::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_actors_film_count['film_count'],
        y=top_actors_film_count['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_film_count['film_count'],
        textposition='inside',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Acteurs ayant joués dans le plus de films',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()

- On veut afficher les acteurs ainsi que le nombre de vote total qu'on reçu les films dans lesquels ils ont joué.

In [7]:
actors_by_votes = df.groupby('person_name')['rating_votes'].sum().reset_index()

top_actors_by_votes = actors_by_votes.sort_values('rating_votes', ascending=False).head(10)[::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_actors_by_votes['rating_votes'],
        y=top_actors_by_votes['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_actors_by_votes['rating_votes'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Top 10 des acteurs dans des films ayant eu le plus de votes',
    xaxis_title='Acteurs',
    yaxis_title='Total des votes'
)

fig.show()
